## Getting Started with the Maxar Geospatial Platform Discovery and Ordering APIs

This sample workflow will guide a user through using the Maxar Geospatial Platform Discovery and Ordering APIs with real world examples. The process will go through:
* Searching STAC items
* Viewing pipelines
* Validating an order
* Checking an order's estimated usage
* Placing an order
* Canceling an order

#### Authentication
For a more detailed view of how to authorize with MGP, please see the *Auth and Token Service* notebook in this collection

In [ ]:
import requests
import json

username = "your.email@address.com"
password= "yourpassword"

url = "https://account.maxar.com/auth/realms/mds/protocol/openid-connect/token"

payload = 'client_id=mgp&username={}&password={}&grant_type=password'.format(username, password)
headers = {'Content-Type': 'application/x-www-form-urlencoded'}

response = requests.request("POST", url, headers=headers, data=payload)

access_token = {'Content-Type': 'application/json', 'Authorization': 'Bearer {}'.format(response.json()['access_token'])}

#### Discover STAC items

The first step in this discovery workflow is to search the available STAC items via Discovery. Some variables will be set up here to make the discovery process more manageable.

In [ ]:
bbox = "-105,40,-104,41"
datetime = "2015-01-01T00:00:00Z/2016-01-01T00:00:00Z"
collections = "wv02"
where = "eo:cloud_cover<20"
limit = 10

A list of all of the ids that match the specified criteria will be iterated through with the first id in that list being set to a variable

In [ ]:
stac_search_url = "https://api.maxar.com/discovery/v1/search?bbox={}&datetime={}&collections={}&where={}&limit={}&" \
                  "orderby=id".format(bbox, datetime, collections, where, limit)

stac_search_response = requests.request("GET", stac_search_url, headers=access_token)

features_list = []
for feature in stac_search_response.json()['features'][:limit]:
    features_list.append(feature['id'])
wv02_feature = features_list[0]
print(wv02_feature)

With a single feature selected, the metadata for that feature can be narrowed down. For this workflow, the supported pipelines for the feature are needed, so the result will be narrowed down to show the pipelines and their associated names.

In [ ]:
stac_item_url = "https://api.maxar.com/discovery/v1/collections/{}/items/{}".format(collections, wv02_feature)

stac_item_response = requests.request("GET", stac_item_url, headers=access_token)

for i in stac_item_response.json()['links']:
    print(i['rel'])
    print(i['href'])

For this workflow, the `map-ready` pipeline will be utilized. With the above cell's output, it is now confirmed that the selected feature can be ordered via the `map-ready` pipeline

If there is unfamiliarity with a pipeline and what is needed for the order to work, the *Get a Pipeline by name* request can be used to determine what information needs to be passed in. The namespace of the pipeline can be found in the `href` link from the above function. It is the section of the link between `pipelines` and the pipeline name, in this case, with a namespace of `imagery` and a name of `map-ready`. With the desired pipeline identified, the response can be narrowed down to show the necessary information needed to place an order.

In [ ]:
pipeline_info_url = "https://api.maxar.com/ordering/v1/pipelines/imagery/map-ready"

pipeline_info_response = requests.request("GET", pipeline_info_url, headers=access_token)

required_info = pipeline_info_response.json()['data']['settings_schema']['required']

print("The required information needed to place an order on this pipeline is: {}".format(required_info))
for info in required_info:
    type_info = pipeline_info_response.json()['data']['settings_schema']['properties'][info]
    print("The type of data for {} is: {}".format(info, type_info))

The response from the above cell shows that the `inventory_ids` and the `customer_description` are required for order placement, and that `inventory_ids` is an array of strings while `customer_description` is a string.

#### Ordering a feature

Before an order can be fully placed, the order needs to be verified that it is formatted properly and will be accepted. The estimated cost of the order is also useful information that can be found. Some variables will be set to make the requests more manageable. The `output_config` variable will be a dictionary where a user can store their S3 information, the notificaitons variable will be a list of a dictionary that will store a user's email information, the settings variable will be a dictionary that contains a user's STAC items and coordinates if a full item is not desired for an order, and the metadata variable will be a dictionary that stores a user's `project_id` (order name).

In [ ]:
namespace = 'imagery'
name = 'map-ready'
output_config = {"amazon_s3": {"bucket": "yourS3BucketName", "prefix": "your/S3/Bucket/Prefix/Name"}}
settings = {
                "inventory_ids": [wv02_feature],
                "customer_description": "your order description",
                "aoi": {
                    "coordinates": [
                        [
                            [-103.991089, 39.546412],
                            [-103.900452, 39.546412],
                            [-103.900452, 39.474365],
                            [-103.991089, 39.474365],
                            [-103.991089, 39.546412]
                        ]
                    ],
                "type": "Polygon"
                }
            }
notifications = [{"type": "email", "target": "your.email@address.com", "level": "FINAL_ONLY"}]
metadata = {"project_id": "your order name"}

The order can now be validated before it is fully placed. *Note: The response can vary by the type of content ordered*

In [ ]:
order_validate_url = "https://api.maxar.com/ordering/v1/pipelines/{}/{}/validate".format(namespace, name)

order_validate_payload = json.dumps({"settings": settings, "output_config": output_config, "notifications": notifications, 
                                     "metadata": metadata})

order_validate_response = requests.request("POST", order_validate_url, headers=access_token, data=order_validate_payload)

print(order_validate_response.json())

The message shows *validation successful*, indicating that the order passes the validation, meaning that the order can be placed. One last thing to check before placing the order is to check the estimated usage of the order. *Note: The response can vary by the type of content ordered*

In [ ]:
order_estimate_url = "https://api.maxar.com/ordering/v1/pipelines/{}/{}/estimate".format(namespace, name)

order_estimate_payload = json.dumps({"settings": settings, "output_config": output_config, "notifications": notifications, 
                                     "metadata": metadata})

order_estimate_response = requests.request("POST", order_estimate_url, headers=access_token, data=order_estimate_payload)

print(order_estimate_response.json())

The response shows that the order will use `62.351 sqkm` of usage. The following cell will calculate what the cost in credits for the order will be. The ID for the product being used in the order will need to be obtained, in this case, for the product `PRODUCT_ARCHIVE_IMAGERY_RECENT_DOWNLOAD`. The sqkm usage amount will also be set to a variable.

In [ ]:
for product_id in order_estimate_response.json()['data']['usage_estimate']:
    desired_product_id = product_id['product_id']
    order_quantity = product_id['quantity']
    print("The ID for the product being utilized in the order is {}".format(desired_product_id))

With the product ID set, this product needs to be found with the associated rate table for the user. The `Tier 1 v1.0  (Default)` rate table will be utilizied for this.

In [ ]:
rate_table_url = "https://api.maxar.com/account-service/api/v1/ratetables"

rate_table_response = requests.request("GET", rate_table_url, headers=access_token)

desired_rate_table = "Tier 1 v1.0  (Default)"

for rate in rate_table_response.json()['rateTables']:
    if rate['name'] == desired_rate_table:
        desired_table_id = rate['id']
print("The ID for the rate table {} is {}".format(desired_rate_table, desired_table_id))

The ID associated with the rate table has now been found. The cost of the order will be calculated by cross referencing the sqkm quantity from the order estimate with the actual credit cost for the product being used in the order.

In [ ]:
rate_product_url = "https://api.maxar.com/account-service/api/v1/ratetables/{}/productCredits".format(desired_table_id)

rate_product_response = requests.request("GET", rate_product_url, headers=access_token)

for rate_product in rate_product_response.json():
    if rate_product['product']['id'] == desired_product_id:
        print("Product {} has a cost of {} credits per {}".format(
            desired_product_id, rate_product['credits'], rate_product['creditUnitType']['description']))
        print("The order placed would consume {} credits".format(order_quantity * rate_product['credits']))

With the order cost calculated, the order can now be placed using the same variables from the order validate and order estimate requests.

In [ ]:
order_url = "https://api.maxar.com/ordering/v1/pipelines/{}/{}/order".format(namespace, name)

order_payload = json.dumps({"settings": settings, "output_config": output_config, "notifications": notifications, 
                            "metadata": metadata})

order_response = requests.request("POST", order_url, headers=access_token, data=order_payload)

print(order_response.json())
order_id = order_response.json()['data']['id']

After the order has been placed, a user may want to check the full details of the submitted order. This can be done with the following request and passing in the order id generated from the placement of the order.

In [ ]:
order_details_url = "https://api.maxar.com/ordering/v1/orders/{}".format(order_id)

order_details_response = requests.request("GET", order_details_url, headers=access_token)

print(order_details_response.json())

A user also may want to occasionally check up on the status of the order. This can be done by utilizing the following request and checking the message of the response.

In [ ]:
order_events_url = "https://api.maxar.com/ordering/v1/orders/{}/events".format(order_id)

order_events_response = requests.request("GET", order_events_url, headers=access_token)

print(order_events_response.json())

In some instances, a user may want to cancel an order. The following request can be ran to cancel the desired order. Orders can be cancelled until the delivery process begins.

In [ ]:
cancel_order_url = "https://api.maxar.com/ordering/v1/order/{}/cancel".format(order_id)

cancel_order_response = requests.request("POST", cancel_order_url, headers=access_token)

print(cancel_order_response.json())